# Criteria

In [6]:
import pandas as pd
import numpy as np

df = pd.read_csv('Monthly_Update.csv')

stimulants = ['Cocaine', 'Methamphetamine', 'Speed', 'Amphetamine']

stimulant_set = df['Sold As/Expected Drug'].str.contains('|'.join(stimulants), case=False, na=False)

stimulant_condition_cocaine = (df['Sold As/Expected Drug'].str.contains('Cocaine', case=False, na=False) &
                               ~df['FTIR Spectrometer'].str.contains('Cocaine', case=False, na=False))

stimulant_condition_meth_speed = (df['Sold As/Expected Drug'].str.contains('Methamphetamine|Speed', case=False, na=False) &
                                  ~df['FTIR Spectrometer'].str.contains('Methamphetamine|Speed', case=False, na=False))

stimulant_condition_amphetamine = (df['Sold As/Expected Drug'].str.contains('Amphetamine', case=False, na=False) &
                                  ~df['FTIR Spectrometer'].str.contains('Amphetamine', case=False, na=False))

stimulant_condition_fentanyl = (df['Fentanyl'] == 'Positive')

stimulant_condition_benzodiazepine = (df['Benzodiazepine'] == 'Positive')

stimulant_condition_levamisole = df['FTIR Spectrometer'].str.contains('Levamisole|Fentanyl|Bezodiazepine', case=False, na=False)

df['Alert'] = 0

stimulant_condition_alert = (stimulant_condition_cocaine | stimulant_condition_meth_speed |
                     stimulant_condition_fentanyl | stimulant_condition_benzodiazepine |
                     stimulant_condition_levamisole | stimulant_condition_amphetamine)

df.loc[stimulant_set & stimulant_condition_alert, 'Alert'] = 1


depressants = ['Fentanyl', 'Benzodiazepine', 'Heroin', 'Morphine']
depressant_set = df['Sold As/Expected Drug'].str.contains('|'.join(depressants), case=False, na=False)
depressant_condition1 = (df['Sold As/Expected Drug'].str.contains('Heroin', case=False, na=False) &
                               ~df['FTIR Spectrometer'].str.contains('Heroin', case=False, na=False))
depressant_condition2 = (df['Sold As/Expected Drug'].str.contains('Morphine', case=False, na=False) &
                               ~df['FTIR Spectrometer'].str.contains('Morphine', case=False, na=False))

depressant_condition3 = (df['FTIR Spectrometer'].str.contains('Xylazine', case=False, na=False) | 
                         df['FTIR Spectrometer'].str.contains('Nitazene', case=False, na=False))

fentanyl_condition = (df['Sold As/Expected Drug'].str.contains('Fentanyl', case=False, na=False) &
                      ~(df['Fentanyl'] == 'Positive') & ~df['FTIR Spectrometer'].str.contains('Fentanyl', case=False, na=False))

benzo_condition = (df['Sold As/Expected Drug'].str.contains('Benzodiazepine', case=False, na=False) &
                   ~(df['Benzodiazepine'] == 'Positive') & ~df['FTIR Spectrometer'].str.contains('Benzodiazepine', case=False, na=False))


depressant_condition_alert = (depressant_condition1 | depressant_condition2 | depressant_condition3 | fentanyl_condition | benzo_condition )
df.loc[depressant_set & depressant_condition_alert, 'Alert'] = 1



condition_unspecified = (df['Fentanyl'] == 'Unspecified') | (df['Benzodiazepine'] == 'Unspecified')

df['Uncertain Alert'] = 0 
df.loc[(df['Alert'] != 1) & (stimulant_set & condition_unspecified), 'Uncertain Alert'] = 1



df['Alert'] = df['Alert'].astype(int)
df['Unified Alert'] = np.where((df['Alert'] == 1) | (df['Uncertain Alert'] == 1), 1, 0)
df['Alert Type'] = 'No Alert'  

df['Alert Type'] = np.where(df['Alert'] == 1, 'Alert', df['Alert Type'])
df['Alert Type'] = np.where(df['Uncertain Alert'] == 1, 'Uncertain Alert', df['Alert Type'])

df_old = pd.read_csv('Dashboard.csv')

df_dashboard = df_old.merge(df, how = 'outer')
df_dashboard = df_dashboard.drop_duplicates()
new_column_order = ['Date','Location','Province','Address','Health Authority','Drug Checking Site','Sold As/Expected Drug','Category','Colour','Texture','Code','Fentanyl','Benzodiazepine','FTIR Spectrometer','Source','Alert','Uncertain Alert','Unified Alert','Alert Type']
df_dashboard = df_dashboard[new_column_order]
df_dashboard = df_dashboard.sort_values(by = 'Date', ascending = False)
df_dashboard['Alert'] = df_dashboard['Alert'].clip(upper=1)
df_dashboard.to_csv('Dashboard.csv', index=False)


/var/folders/mk/3rm9f8jj0_54616_xx1g__n00000gn/T/ipykernel_86848/1274189169.py:70: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_old = pd.read_csv('Dashboard.csv')
